РЕКОМЕНДАТЕЛЬНАЯ СИСТЕМА

Цели и задачи проекта:  
1.вывести сочетания товаров в чеках, которые покупаются вместе с целью предложения данного ассоримента покупателям, расширения группы комплиментарных товаров  
2.контроль за наличием на полке ходовых товаров  
3.поиск покупательских инсайтов

In [2]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

from apyori import apriori
import apyori

import seaborn as sns


1.выгрузка дата сета с помощью sql

DECLARE @iDateStart int = 20210301
		,@iDateEnd int = 20210331

DROP TABLE IF EXISTS #filtReceipts

SELECT fk_ReceiptID
INTO #filtReceipts
FROM [DWH_FM].[dbo].[v_FactSales] as One
LEFT JOIN v_DimStores_MK as Two on One.fk_storeMkID = two.sk_storeMkID
WHERE 1=1
	and OurFirm = 'МК' 
	and WarehouseGroup = 'ТО Ижевск' 
	and fk_Calendar between @iDateStart and @iDateEnd
GROUP BY fk_ReceiptID
HAVING COUNT(*) > 1


SELECT One.fk_ReceiptID, bk_Product, Quantity
FROM [DWH_FM].[dbo].[v_FactSales] as One
LEFT JOIN v_DimStores_MK as Two on One.fk_storeMkID = two.sk_storeMkID
JOIN #filtReceipts f on f.fk_ReceiptID =  One.fk_ReceiptID
WHERE 1=1
	and OurFirm = 'МК' 
	and WarehouseGroup = 'ТО Ижевск' 
	and fk_Calendar between @iDateStart and @iDateEnd
Order by fk_ReceiptID


я сначала удаляю временную таблицу #filt 
потом заполняю ее список заказов у которых более одной строки
потом использую ее в качестве фильтра в основном запросе, то есть при иннер джойне на нее остаются только заказы с более чем одной строкой

убираем чеки с 1 товаром, ограничиваем одним месяцем и городом

2.загружаем полученный датасет

In [3]:
dataset = pd.read_excel('Diplom_dataset6.xlsx')
# посомтрим на датасет
dataset.head()

,Check,SKU,Volume
0,23420273,PR0016291,1.000
1,23420273,ПЖП1009707,3.664
2,23420273,АКС0002176,1.000
3,23420273,ПЖП1018128,1.156
4,23420292,АКС0000969,10.000


3.Смотрим форматы и заполненность значениями датасета

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496595 entries, 0 to 496594
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Check   496595 non-null  int64  
 1   SKU     496595 non-null  object 
 2   Volume  496595 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 11.4+ MB


4.Считаем число уникальных номенклатур в датасете

In [5]:
dataset.SKU.value_counts()

PR0016291     49483
ПЖП1009932    23452
ПЖП0100032    13328
ПЖП0102643    12541
PR0003099      8498
              ...  
ПЖП33236          1
PR0016738         1
PR0015667         1
PR0014260         1
ПЖП1009214        1
Name: SKU, Length: 905, dtype: int64

Выводы:


5.трансформируем датасет для модели

In [6]:
good = dataset.groupby('Check')['SKU'].apply(lambda r: ' '.join([str(A) for A in r]))

In [7]:
good.head(10)

Check
23420273           PR0016291 ПЖП1009707 АКС0002176 ПЖП1018128
23420292                                АКС0000969 ПЖП1018601
23420295            PR0002349 PR0016291 АКС0001983 ПЖП1008987
23420310    ПЖП1009925 PR0006575 ПЖП1020792 PR0004894 ПЖП1...
23420322                                 ПЖП1020792 PR0014096
23420330                                  PR0016408 PR0003099
23420332                       PR0003722 АКС0000969 PR0016291
23420342                                ПЖП1009927 ПЖП1009887
23420346                                АКС0001761 ПЖП1009932
23420348                                  PR0009191 PR0001513
Name: SKU, dtype: object

6.Применяем модель, настраиваем критерии вывода правил

In [8]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.001, 
                                   min_confidence=0.01, min_lift=1.05, 
                                   min_length=1)

In [9]:
association_rules

<generator object apriori at 0x000001D79798BD60>

In [10]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

7.смотрим на получившиеся правила

In [11]:
asr_df

,from,to,confidence,support,lift
0,PR0001320,PR0016291,0.393192,0.00403132,1.57094
1,PR0001334,PR0016291,0.396209,0.00222051,1.583
2,PR0001729,ПЖП1009932,0.139789,0.00181081,1.17843
3,PR0003099,PR0006575,0.024947,0.00107232,1.09651
4,PR0003099,PR0010747,0.0354201,0.00152249,1.07336
...,...,...,...,...,...
121,ПЖП1009932,ПЖП1014429,0.0248593,0.00294888,1.21531
122,ПЖП1009932,ПЖП1020791,0.0106174,0.00125947,1.23548
123,ПЖП0100030,PR0016291 ПЖП0100647,0.0271586,0.00107232,5.50134
124,ПЖП0100030,ПЖП1018601 PR0016291,0.0253651,0.00100151,4.54233


8.выбираем топ 10правил по метрике support
vожно охарактеризовать этот показатель как индекс частоты встречаемости конкретного продукта в имеющихся транзакциях.
насколько часто какие-то два и более продукта встречаются вместе

In [16]:
asr_df.sort_values('support').tail(10)

,from,to,confidence,support,lift
91,PR0016291,ПЖП1021055,0.0187135,0.00468382,1.32748
79,PR0016291,ПЖП1008979,0.0195219,0.00488614,1.38981
76,PR0016291,ПЖП0100647,0.0197239,0.00493672,1.29079
74,PR0016291,АКС0002210,0.0204111,0.0051087,1.11688
87,PR0016291,ПЖП1018601,0.0223107,0.00558416,1.1439
30,PR0006575,PR0016291,0.290129,0.00660084,1.15917
75,PR0016291,ПЖП0100030,0.0425803,0.0106575,1.07843
83,PR0016291,ПЖП1010554,0.0427217,0.0106929,1.34301
5,PR0003099,PR0016291,0.272417,0.0117095,1.0884
77,PR0016291,ПЖП0102643,0.0916072,0.0229284,1.44414


9.Подгружаем датасет с наименованием товара для пользовательской идентификации получившихся правил

In [17]:
book = pd.read_excel('book.xlsx')
# посомтрим на датасет
book.head()

,SKU,SKU_name
0,PR0004762,Скумбрия б/г х/к 2 сорт 1*3 ПятыйПрич
1,PR0011875,Набор к пиву Горбуша х/к фасованный вес. Пятый...
2,PR0018247,Салака калтычки х/к*2 ПятыйПрич
3,ПЖП1018708,"Минеральная вода Увинская Жемчужина газир 1,5л..."
4,АКС0001331,ВСТ Консервы ветчина Московская ж/б 325г*36


10.применяем загруженный датасет

In [18]:
def get_rule_title(rule):
    print(book[book.SKU.isin(rule['from'].split(' '))]['SKU_name'].values)
    print('----------->')
    print(book[book.SKU == rule['to']]['SKU_name'].values)

In [19]:
get_rule_title(asr_df.loc[91])

['Пакет-майка БОЛЬШАЯ']
----------->
['ВСТ Окорок охл в/у (ок-2кг) СМК']


Промежуточные выводы:  
Видим, что самый популярный товар - пакет. Ожидаемо. Очистим датасет от данного товара, так как это не цель исследования.

In [20]:
dataset2 = dataset.loc[dataset['SKU'] != 'PR0016291']
# посомтрим на датасет
dataset2.head()

,Check,SKU,Volume
1,23420273,ПЖП1009707,3.664
2,23420273,АКС0002176,1.000
3,23420273,ПЖП1018128,1.156
4,23420292,АКС0000969,10.000
5,23420292,ПЖП1018601,0.786


In [21]:
dataset2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447112 entries, 1 to 496594
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Check   447112 non-null  int64  
 1   SKU     447112 non-null  object 
 2   Volume  447112 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 13.6+ MB


In [22]:
good2 = dataset2.groupby('Check')['SKU'].apply(lambda r: ' '.join([str(A) for A in r]))

In [23]:
good2.head(10)

Check
23420273                     ПЖП1009707 АКС0002176 ПЖП1018128
23420292                                АКС0000969 ПЖП1018601
23420295                      PR0002349 АКС0001983 ПЖП1008987
23420310    ПЖП1009925 PR0006575 ПЖП1020792 PR0004894 ПЖП1...
23420322                                 ПЖП1020792 PR0014096
23420330                                  PR0016408 PR0003099
23420332                                 PR0003722 АКС0000969
23420342                                ПЖП1009927 ПЖП1009887
23420346                                АКС0001761 ПЖП1009932
23420348                                  PR0009191 PR0001513
Name: SKU, dtype: object

In [24]:
association_rules2 = apyori.apriori(good2.apply(lambda r: r.split(' ')), 
                                   min_support=0.001, 
                                   min_confidence=0.01, min_lift=1.05, 
                                   min_length=1)

In [25]:
association_rules2

<generator object apriori at 0x000001D797B09580>

In [26]:
asr_df2 = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules2:
    pair = item[0] 
    items = [x for x in pair]
    asr_df2.loc[len(asr_df2), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

In [27]:
asr_df2

,from,to,confidence,support,lift
0,PR0001729,ПЖП1009932,0.139789,0.00181101,1.1783
1,PR0003099,PR0006575,0.024947,0.00107244,1.09638
2,PR0003099,PR0010747,0.0354201,0.00152266,1.07324
3,PR0003099,PR0016409,0.053542,0.0023017,3.95966
4,PR0003099,АКС0002210,0.0304778,0.0013102,1.66755
...,...,...,...,...,...
80,ПЖП1009925,ПЖП1009932,0.127063,0.00194759,1.07103
81,ПЖП1009925,ПЖП1016817,0.107591,0.00164913,15.2135
82,ПЖП1009932,ПЖП1010107,0.0356473,0.00422906,1.33892
83,ПЖП1009932,ПЖП1014429,0.0248593,0.00294921,1.21518


In [28]:
asr_df2.sort_values('support').tail(10)

,from,to,confidence,support,lift
48,PR0014096,PR0016409,0.0995844,0.00303015,7.3647
63,ПЖП0100030,ПЖП0100647,0.0778888,0.00307568,5.09668
67,ПЖП0100030,ПЖП1018601,0.0849347,0.00335391,4.35422
42,PR0011871,ПЖП0100032,0.0847018,0.00347025,1.25629
19,PR0006001,ПЖП1009932,0.150759,0.00366754,1.27077
53,PR0015011,ПЖП1009932,0.145817,0.00377378,1.22911
78,ПЖП1009917,ПЖП1009932,0.131413,0.00390024,1.1077
23,PR0006575,PR0010747,0.172743,0.00393059,5.2342
82,ПЖП1009932,ПЖП1010107,0.0356473,0.00422906,1.33892
75,ПЖП1009879,ПЖП1009932,0.143415,0.00464387,1.20886


In [30]:
get_rule_title(asr_df2.loc[48])

['Фарш Свиной охл лоток 400гр ВСТ СМК']
----------->
['ФМ Фарш из говядины 400 гр']


11.выводы 
Потребитель предпочитает самостоятельно смешавать фарш: свинина + говядина. Необходимо смотреть за наличием данных товаров на полке
